# AWS Virtual Private Cloud

A VPC is a logically isolated networking environment for your AWS resources. In this notebook we'll create a VPC, and partition it into public and private subnets, showing how we can use subnets as security containers or boundaries.

## Creating a VPC

In [ ]:
import boto3

client = boto3.client('ec2')
vpc_response = client.create_vpc(
    CidrBlock='10.0.0.0/16'
)

vpcId = vpc_response['Vpc']['VpcId']

print vpc_response

## Subnets

Now that we have a VPC, we can create subnets. In contrast to a VPC, which spans all the availability zones in
a region, a subnet is specific to an availability zone.

So the first thing to figure out is what are the availability zones in the region.

In [ ]:
response = client.describe_availability_zones()
availability_zones = response['AvailabilityZones']

for az in availability_zones:
    print az

One we know the zones, we can figure out if we can create a subnet in the zone by doing a dry run.

TO DO: try this code in us-west-1, which has 3 AZs with only two of those supporting VPC subnet creation. I can create subnets in all AZs in us-east-1, so I just flip DryRun to false in the code below.

In [ ]:
# Example code to create a subnet in each AZ
subnet_no = -1
cidr_base = '10.0.'
for az in availability_zones:
    subnet_no += 1
    zone_name = az['ZoneName']
    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    try:
        response = client.create_subnet(
            DryRun=True,
            VpcId = vpcId,
            CidrBlock = cidr_base + str(subnet_no) + '.0/24',
            AvailabilityZone = zone_name
        )

        print response
        
        
    except Exception, Argument:
        print Exception
        print Argument
        
        


In [ ]:
# For the rest of this work book we'll create two subnets in two AZs. We'll treat 
# the odds as public, the evens as private.

availability_zones = availability_zones[0:2]
public_subnets = []

subnet_no = -1
cidr_base = '10.0.'
for i in range(2):
    subnet_no += 1
    zone_name = availability_zones[i]['ZoneName']
    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )
    
    public_subnets.append(response['Subnet']['SubnetId'])
    

    subnet_no += 1    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )    
   
    


In [ ]:
# We want public addresses on instances launched in the public subnet
# by default.

print public_subnets

for id in public_subnets:
    response = client.modify_subnet_attribute(
        SubnetId=id,
        MapPublicIpOnLaunch={
            'Value': True
        }
    )
    
    print response

## Public Subnets

What makes a subnet public? Having Internet routable IP addresses, and access to the Internet. We've made the
default address option for instances in our public subnets public/routable. To allow Internet access, we create and attach an internet gateway to the VPC, the create a route table with routes from the public subnets to the Internet.

In [ ]:
## Create an internet gateway
igw_response = client.create_internet_gateway()
igwId = igw_response['InternetGateway']['InternetGatewayId']
print 'Created gateway ', igwId

In [ ]:
# Attach internet gateway to VPC
response = client.attach_internet_gateway(
    InternetGatewayId=igw_response['InternetGateway']['InternetGatewayId'],
    VpcId=vpcId
)

print response

Instead of adding an external route to the default route table associated with 
the VPC, we'll create a route table for explicitly controlling subnet access
to the internet. Note that the default route table allows traffic within the 
VPC, and is implicitly associated with the subnets.

In [ ]:
# Create route table for public subnets
rt_response = client.create_route_table(
    VpcId = vpcId
)

routeTableId = rt_response['RouteTable']['RouteTableId']
print 'Create route table ', routeTableId

In [ ]:
# Add route to the interwebs from the gateway

response = client.create_route(
    RouteTableId=routeTableId,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igwId,
)

print response

In [ ]:
# Associate the subnets with the route table
associationIds = []

for subnet in public_subnets:
    response = client.associate_route_table(
        SubnetId=subnet,
        RouteTableId=routeTableId
    )
        
    associationIds.append(response['AssociationId'])

## Interlude - Launch an Instance in a Public Subnet

Now that we have a VPC with public subnets, we should be able to launch an instance into a public subnet and
connect to it. To allow connectivity to be verified, we'll use user data to install a LAMP stack with a test
page we can retrieve.

First, we'll need a security group allowing ingres

In [ ]:
#Create security group
response = client.create_security_group(
    GroupName='web_sg',
    Description='Use for launching public web service',
    VpcId=vpcId
)

webSgID = response['GroupId']
print 'created security group ', webSgID

In [ ]:
# Clean up
response = client.delete_security_group(
    GroupId=webSgID
)

print response

## Clean Up

In [ ]:
# Disassociate the public subnets from the route table
for association in associationIds:
    response = client.disassociate_route_table(
    AssociationId=association
)

In [ ]:
# Delete the route table
response = client.delete_route_table(
    RouteTableId = routeTableId
)

In [ ]:
# Clean up gateway
response = client.detach_internet_gateway(
    InternetGatewayId=igwId,
    VpcId=vpcId
)
print response

response = client.delete_internet_gateway(
    InternetGatewayId=igwId
)
print response

In [ ]:
# Delete subnets
response = client.describe_subnets(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                vpcId,
            ]
        },
    ]
)

subnets = response['Subnets']

for sn in subnets:
    response = client.delete_subnet(
        SubnetId=sn['SubnetId']
    )
    
    print response

# Delete the VPC
response = client.delete_vpc(
    VpcId = vpcId
)

print response